In [1]:
using Clustering
using HDF5
using LinearAlgebra
using Dates
using Threads

LoadError: ArgumentError: Package Threads not found in current path:
- Run `import Pkg; Pkg.add("Threads")` to install the Threads package.


In [2]:
function get_keys(df)
    h5open(df, "r") do file
        return keys(file)
    end
end

function hankel(y::AbstractArray)
    m, time_duration = size(y) # m - dimention of output vector y, time_duration - length of timeseries (number of time steps)
    q = Int(round(time_duration/2)) # q - is the size of Hankel matrix 
    H = zeros(eltype(y), q * m , q) 
    for r = 1:q, c = 1:q # r - rows, c -columns
        H[(r-1)*m+1:r*m, c] = y[:, r+c-1]
    end
    return H, m
end

function lsid_ACx0S(Y::AbstractArray, Δt) #, δ = 1e-6)
    # y - output time series dim[y] = m x number_of_time_steps
    # δ - precission cutoff all the smaller values of Σ will be discarded 
    
    H, m = hankel(Y) # Hankel matrix and dimention of output (should be 12 in our case)
    U, Σ, Vᵈ = svd(H) # Singular value decomposition of H to U,  Σ,  V†
    
    s = Diagonal(sqrt.(Σ)) # Matrix square root 
    U = U * s
    Vᵈ = s * Vᵈ
     
    # n = argmin(abs.(Σ/maximum(Σ) .- δ)) - 1 # estimated rank of the system

    Σ_log = log.(Σ/maximum(Σ))
    Σ²ᴰ = reshape(Σ_log, (1, length(Σ_log)))

    n = minimum(counts(kmeans(Σ²ᴰ, 2))) + 1
    
    C = U[1:m, 1:n] # m - dimention of output, n - rank of the system
    
    U_up = U[1:end-m, :] # U↑
    U_down = U[m+1:end, :] # U↓
    
    A = pinv(U_up) * U_down
    Ac = log(A)/Δt 
    
    Ac = Ac[1:n, 1:n] 
    A = A[1:n, 1:n] # n - estimated rank of the system
    
    x₀ = pinv(U) * H
    x₀ = x₀[1:n, 1]
    
    return A, Ac, C, x₀, n, Σ²ᴰ # was A, Ac, C, x0

end

function get_rho_series(file_name, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[string(γ)])
        tᵧ = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ = []
        t = []

        for i in 1:length(tᵧ)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            
            push!(ρ, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t, convert(Float64, tᵧ[i]))
        end
        return(ρ, t)
    end
end

function get_vec_rho_series(file_name, γ)
    h5open(file_name, "r") do file
        ρ = read(file[string(γ)])
        ρ₀₀ = ρ["p0"]; Re_ρ₀₁ = ρ["s_re"];  Im_ρ₀₁ = ρ["s_im"]
        return([ρ₀₀;; Re_ρ₀₁;; Im_ρ₀₁], ρ["t"])
    end
end

function get_Y(directory, γᵢ)
    
    ρᵉᵥ, tᵉ = get_vec_rho_series(directory*"State_B1_data.h5", string(γᵢ))
    ρᵍᵥ, tᵍ = get_vec_rho_series(directory*"State_B2_data.h5", string(γᵢ))
    ρˣᵥ, tˣ = get_vec_rho_series(directory*"State_B3_data.h5", string(γᵢ))
    ρʸᵥ, tʸ = get_vec_rho_series(directory*"State_B4_data.h5", string(γᵢ))
    
    lᵉ = size(ρᵉᵥ, 1); lᵍ = size(ρᵍᵥ, 1); lˣ = size(ρˣᵥ, 1); lʸ = size(ρʸᵥ, 1)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series
    
    @assert(tᵉ[ lᵐᵃˣ ] == tᵍ[ lᵐᵃˣ ] == tˣ[ lᵐᵃˣ ] == tʸ[ lᵐᵃˣ ])
    
    Y = [ρᵉᵥ[1:lᵐᵃˣ, :];; ρᵍᵥ[1:lᵐᵃˣ, :];; ρˣᵥ[1:lᵐᵃˣ, :];; ρʸᵥ[1:lᵐᵃˣ, :]]
    
    return transpose(Y), tᵉ[ 1:lᵐᵃˣ]
end

get_Y (generic function with 1 method)

In [3]:
# Names of files with Kurt data
basis_file_names = ["State_B"*string(n) for n=1:4]
println(basis_file_names)
dodeca_file_names = ["State_D"*string(n) for n=1:20]
print(dodeca_file_names)
#directory = "C:/Users/Zakhar/Documents/GitHub/JPOP_SID/DATA/"
directory = "C:/Users/Zakhar/Documents/GitHub/Kurt2021/2022JAN24/DATA/"

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
res_file_name = "Kurt_LSID_ACx0S_" * date_and_time_string * ".h5"

γ = get_keys(directory*basis_file_names[1]*"_data.h5")

["State_B1", "State_B2", "State_B3", "State_B4"]
["State_D1", "State_D2", "State_D3", "State_D4", "State_D5", "State_D6", "State_D7", "State_D8", "State_D9", "State_D10", "State_D11", "State_D12", "State_D13", "State_D14", "State_D15", "State_D16", "State_D17", "State_D18", "State_D19", "State_D20"]

8-element Vector{String}:
 "0.079477"
 "0.25133"
 "0.79477"
 "2.5133"
 "25.133"
 "251.33"
 "7.9477"
 "79.477"

In [4]:
γ = ["79.477", "251.33"]

lck_read  = ReentrantLock() # create lock to use when reading in the loop below
lck_write = ReentrantLock() # create lock to use when writing in the loop below

Y = []; Δt = []

for i in length(γ)
    Yᵢ, tᵢ = get_Y(directory, γ[i])
    @assert maximum(diff(tᵢ)) ≈ minimum(diff(tᵢ))
    Δtᵢ = tᵢ[2]-tᵢ[1]
    push!(Y, Yᵢ)
    push!(Δt, Δtᵢ)
end # of reading loop
    
        

    

@time Threads.@threads for i in length(γ)
    
    γᵢ = γ[i]
    println("| SID for γ = "*γᵢ*"...")
      
    A, Ac, C, x₀, n, Σ²ᴰ = lsid_ACx0S(Y[i], Δt[i])   
    
    println(" SID for γ ="*string(γᵢ)*"done.|")
    
    lock(lck_write)
            
        try

            h5open(directory*res_file_name,"cw") do fid  
            # read-write, create file if not existing, preserve existing contents

                γ_group = create_group(fid, "gamma_"*string(γᵢ))

                γ_group["A"] = convert.(ComplexF64, A)
                γ_group["Ac"] = convert.(ComplexF64, Ac)
                γ_group["C"] = convert.(ComplexF64, C)
                γ_group["x0"] = convert.(ComplexF64, x₀)
                γ_group["n"] = n
                γ_group["sigma"] = Σ²ᴰ

            end # of HDF5 writing

        finally
            unlock(lck_write)
        end

        println(" Saving for γ ="*string(γᵢ)*"done.|")
    
end # of loop over the coupling levels γ


| SID for γ = 79.477...


LoadError: TaskFailedException

[91m    nested task error: [39mUndefVarError: t not defined
    Stacktrace:
     [1] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m.\[39m[90m[4mIn[4]:17[24m[39m[90m [inlined][39m
     [2] [0m[1m(::var"#39#threadsfor_fun#12"{Vector{String}})[22m[0m[1m([22m[90monethread[39m::[0mBool[0m[1m)[22m
    [90m   @ [39m[35mMain[39m [90m.\[39m[90m[4mthreadingconstructs.jl:85[24m[39m
     [3] [0m[1m(::var"#39#threadsfor_fun#12"{Vector{String}})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @ [39m[35mMain[39m [90m.\[39m[90m[4mthreadingconstructs.jl:52[24m[39m

In [5]:
function get_kurt_lsid(file, γ_group)

    h5open(file,"r") do fid # read-only
        A = read(fid[γ_group]["A"])
        Ac = read(fid[γ_group]["Ac"])
        C = read(fid[γ_group]["C"])
        x₀ = read(fid[γ_group]["x0"])
        n = read(fid[γ_group]["n"])
        Σ = read(fid[γ_group]["sigma"])
        
        return A, Ac, C, x₀, n, Σ
    end
end

get_kurt_lsid (generic function with 1 method)

In [6]:
A, Ac, C, x₀, n, Σ²ᴰ = get_kurt_lsid(directory*res_file_name, "gamma_"*"251.33") 

LoadError: unable to determine if C:/Users/Zakhar/Documents/GitHub/Kurt2021/2022JAN24/DATA/Kurt_LSID_ACx0S_2022-Sep-29_at_21-02.h5 is accessible in the HDF5 format (file may not exist)

In [7]:
A 

LoadError: UndefVarError: A not defined